#### Importing libraries

In [2]:
import numpy as np
import pandas as pd
# re le regular expression ho ki nai vancha
import re
# delete like the for of in with through stopwords
from nltk.corpus import stopwords
# porterstemmer le loving =love , eating = eat
from nltk.stem.porter import PorterStemmer 
# vector from [0,1]
from sklearn.feature_extraction.text import TfidfVectorizer
# for to split the data
from sklearn.model_selection import train_test_split
# fake news is classification problem so we use logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
import nltk

# importing dataset as fake and real

In [4]:
data_fake = pd.read_csv('fake.csv')
data_real = pd.read_csv('real.csv')

In [5]:
data_real.head()

,Title,Description
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...


In [6]:
data_fake.head()

,Title,Description
0,ट्रम्पडम: ट्रम्पको विदेश नीतिको जिज्ञासु संसा...,निरज श्रीवास्तव 21st Century WireIt डोनाल्ड ट्...
1,BREAKING: वफादार ट्रम्प समर्थकले महान्यायाधिवक...,"सेन जेफरसन ब्यूरेगार्ड सेसन्स III, डोनाल्ड ट्र..."
2,चीन ड्रोन ड्रामा ट्रम्पको पहिलो ट्वीट भन्दा पह...,डोनाल्ड ट्रम्पले अमेरिकी अन्डरवाटर ड्रोन जफत ग...
3,एक रक्षात्मक जोन केरीले सोध्छ कि इजरायलले आफ्न...,जोन केरीले ओबामा प्रशासन इजरायलको मित्र भएको द...
4,Reince Priebus ट्रम्पको दाबीलाई समर्थन गर्दछ क...,डोनाल्ड ट्रम्पले स्वतन्त्र प्रेस बिरूद्ध युद्ध...


# clasify gareu data fake lai 0 le real lai 1 lw

In [8]:
data_fake["class"]= 0
data_real["class"]= 1

# 2 tai data lai concatinate gareu

In [10]:
data = pd.concat([data_real,data_fake],axis=0)
data.head()

,Title,Description,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...,1


# title lai delete gareko 

In [12]:
cleaned_data= data.drop(['Description'],axis=1, inplace=False)
cleaned_data.head(10)

,Title,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,1
5,घाँस खेतीबाट वार्षिक २० लाख रुपियाँ गोकर्णकाे ...,1
6,वैदेशिक रोजगारीका लागि मलेसिया प्रमुख गन्तव्य,1
7,"युक्रेनीलाई बिजुली, पानीबाट वञ्चित गराउँदै रूस",1
8,सुनौलो सहकारीमा पुनः सापकोटा,1
9,अभ्यास खेलमा नेपाल पराजित,1


# suffle gareu data lai

In [14]:
suffle_data = cleaned_data.sample(frac=1)
suffle_data.head(10)

,Title,class
458,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0
5540,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1
3561,भाषा विस्तारमाथि बहस,1
4644,दैनिक ज्याला आठ किलो धान!,1
3525,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1
2565,अफगानिस्तानलाई थप ४० मिलियन अमेरिकी डलर नगद सह...,1
2387,अर्थतन्त्रका बाह्य पक्षमा उल्लेख्य सुधार,1
816,अस्ट्रेलियामा नेपाली कलाकारको धमाका,1
9427,ब्याडमिन्टनमा दीपेश र रशिला च्याम्पियन,1
2997,विवादास्पद फक्सका पूर्व सीईओ रोजर आइल्सको ७७ व...,0


In [15]:
suffle_data.reset_index(inplace=True)
suffle_data.head()

,index,Title,class
0,458,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0
1,5540,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1
2,3561,भाषा विस्तारमाथि बहस,1
3,4644,दैनिक ज्याला आठ किलो धान!,1
4,3525,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1


In [16]:
suffle_data.drop(['index'],axis=1,inplace = True)
suffle_data.head(5)

,Title,class
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0
1,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1
2,भाषा विस्तारमाथि बहस,1
3,दैनिक ज्याला आठ किलो धान!,1
4,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1


# checking null value

In [18]:
suffle_data.isnull().sum()

Title    0
class    0
dtype: int64

# mixing both fake and real into 1

In [20]:
suffle_data.to_csv('mixed.csv', index=False)

In [21]:
data = pd.read_csv('mixed.csv')
data.head()

,Title,class
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0
1,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1
2,भाषा विस्तारमाथि बहस,1
3,दैनिक ज्याला आठ किलो धान!,1
4,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1


# Preprocessing

In [23]:
import re
import string

def wordopt(Title):
    
    if not isinstance(Title, str):  # Handle non-string inputs
        return Title
    # Title = Title.lower()
    Title = re.sub(r'[\[\]]', '', Title)  # Remove square brackets
    Title = re.sub(r'https?://\S+|www\.\S+', '', Title)  # Remove URLs
    Titel = re.sub(r'<.*?>', '', Title)  # Remove HTML tags
    Title = re.sub(f'[{re.escape(string.punctuation)}]', '', Title)  # Remove punctuation
    Title = re.sub(r'\n', '', Title)  # Remove newlines
    Title = re.sub(r'\w*\d\w*', '', Title)  # Remove words with digits
    # Description = re.sub(r'\s+', ' ', Description).strip()  # Normalize whitespace
    
    return Title


In [24]:
data['Title2']= data['Title'].apply (wordopt)
data.head(10)

,Title,class,Title2
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...
1,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1,विश्वकप फाइनल फ्रान्सका राष्ट्रपति म्याक्रोँ क...
2,भाषा विस्तारमाथि बहस,1,भाषा विस्तारमाथि बहस
3,दैनिक ज्याला आठ किलो धान!,1,दैनिक ज्याला आठ किलो धान
4,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1,केबीसी को पहिलो एपिसोडको प्रोमो सार्वजनिक
5,अफगानिस्तानलाई थप ४० मिलियन अमेरिकी डलर नगद सह...,1,अफगानिस्तानलाई थप मिलियन अमेरिकी डलर नगद सहाय...
6,अर्थतन्त्रका बाह्य पक्षमा उल्लेख्य सुधार,1,अर्थतन्त्रका बाह्य पक्षमा उल्लेख्य सुधार
7,अस्ट्रेलियामा नेपाली कलाकारको धमाका,1,अस्ट्रेलियामा नेपाली कलाकारको धमाका
8,ब्याडमिन्टनमा दीपेश र रशिला च्याम्पियन,1,ब्याडमिन्टनमा दीपेश र रशिला च्याम्पियन
9,विवादास्पद फक्सका पूर्व सीईओ रोजर आइल्सको ७७ व...,0,विवादास्पद फक्सका पूर्व सीईओ रोजर आइल्सको वर्...


In [25]:
data.columns

Index(['Title', 'class', 'Title2'], dtype='object')

# Tokenization

In [27]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
data['tokens1'] = data['Title2'].apply(word_tokenize)
data.head()

,Title,class,Title2,tokens1
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,"[ओबामाले, अल्टोन, स्टर्लिङको, अन्त्येष्टिमा, प..."
1,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1,विश्वकप फाइनल फ्रान्सका राष्ट्रपति म्याक्रोँ क...,"[विश्वकप, फाइनल, फ्रान्सका, राष्ट्रपति, म्याक्..."
2,भाषा विस्तारमाथि बहस,1,भाषा विस्तारमाथि बहस,"[भाषा, विस्तारमाथि, बहस]"
3,दैनिक ज्याला आठ किलो धान!,1,दैनिक ज्याला आठ किलो धान,"[दैनिक, ज्याला, आठ, किलो, धान]"
4,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1,केबीसी को पहिलो एपिसोडको प्रोमो सार्वजनिक,"[केबीसी, को, पहिलो, एपिसोडको, प्रोमो, सार्वजनिक]"


# acessing the list of stopwords

In [29]:
stop = open('stopwords.txt', encoding='utf-8')  # Specify encoding
stopwords = []
for x in stop:
    x = x.replace('\n', '')  # Remove newline characters
    stopwords.append(x)      # Add to the stopwords list
stop.close()  # Close the file after reading
data.head(1)

,Title,class,Title2,tokens1
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,"[ओबामाले, अल्टोन, स्टर्लिङको, अन्त्येष्टिमा, प..."


In [30]:
stopwords

['',
 'आफ्न',
 'गर्नेछौ',
 'वाहे',
 'जसम',
 'सङ्गको',
 'चाहे',
 'जस्तै',
 'भनेर',
 'तिमीसँ',
 'चाहन्छौ',
 'जाहि',
 'सम्भ',
 'भन्छन्',
 'त्यस्तो',
 'प्रत्येक',
 'कत',
 'थरि',
 'त्यस्तै',
 'कति',
 'गर्छ',
 'आफैल',
 'सायद',
 'त्यसै',
 'सुरुमै',
 'भीत्र',
 'पटक',
 'अलग',
 'अक्स',
 'गर्दछ',
 'जसको',
 'नत्र',
 'राख्',
 'थिएन',
 'छु',
 'यद',
 'हो',
 'अर्थात',
 'गर्द',
 'छौं',
 'पाँ',
 'होकि',
 'भन्न',
 'दि',
 'नै',
 'पछिल्ल',
 'सध',
 'आत्',
 'पक्क',
 'तेस्कार',
 'पर्नेमा',
 'कम से क',
 'एउटा',
 'सक्नुहुन्छ',
 'भन्द',
 'उनीहरु',
 'म',
 'उसलाई',
 'रे',
 'सम',
 'बिच',
 'त्यत्तिकै',
 'जस्त',
 'पनि',
 'यसैल',
 'वाहेक',
 'ठाउँमा',
 'प्रत्ये',
 'गय',
 'अल',
 'निर्दिष्',
 'कोही',
 'यस',
 'आफैलाई',
 'द्वार',
 'यस्तै',
 'अर्क',
 'मैल',
 'एकद',
 'छिन्',
 'कस्त',
 'देखेको',
 'समयम',
 'तिनै',
 'चाले',
 'पर्याप्त',
 'सम्म',
 'जस',
 'स्पष्',
 'यस्त',
 'अरू',
 'दश',
 'समेत',
 'सोध्',
 'त्य',
 'ऊ',
 'तसरी',
 'मात्र',
 'जबकि',
 'तिनीहरूले',
 'किनभने',
 'उदाहरण',
 'नभएको',
 'केही',
 'साय',
 'तिनीहरूलाई',
 'बाहि

# Removing stopwords

In [32]:
def remove_stopwords(x):
    return ",".join([word for word in str(x).split() if word not in stopwords])
data['title3']= data['tokens1']. apply(lambda x :remove_stopwords(x))
data.head(5)

,Title,class,Title2,tokens1,title3
0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,0,ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...,"[ओबामाले, अल्टोन, स्टर्लिङको, अन्त्येष्टिमा, प...","['ओबामाले',,'अल्टोन',,'स्टर्लिङको',,'अन्त्येष्..."
1,विश्वकप फाइनल: फ्रान्सका राष्ट्रपति म्याक्रोँ ...,1,विश्वकप फाइनल फ्रान्सका राष्ट्रपति म्याक्रोँ क...,"[विश्वकप, फाइनल, फ्रान्सका, राष्ट्रपति, म्याक्...","['विश्वकप',,'फाइनल',,'फ्रान्सका',,'राष्ट्रपति'..."
2,भाषा विस्तारमाथि बहस,1,भाषा विस्तारमाथि बहस,"[भाषा, विस्तारमाथि, बहस]","['भाषा',,'विस्तारमाथि',,'बहस']"
3,दैनिक ज्याला आठ किलो धान!,1,दैनिक ज्याला आठ किलो धान,"[दैनिक, ज्याला, आठ, किलो, धान]","['दैनिक',,'ज्याला',,'आठ',,'किलो',,'धान']"
4,'केबीसी १४' को पहिलो एपिसोडको प्रोमो सार्वजनिक,1,केबीसी को पहिलो एपिसोडको प्रोमो सार्वजनिक,"[केबीसी, को, पहिलो, एपिसोडको, प्रोमो, सार्वजनिक]","['केबीसी',,'को',,'पहिलो',,'एपिसोडको',,'प्रोमो'..."


# Stemming the language

In [34]:
# test_words = ["नेपालीको", "आफ्नै", "स्तेम्मेर", "यसलाई", "अनुमोदनको"]
# stemmed_words = nepali_stemmer.stemWords(test_words)
# print("Original Words:", test_words)
# print("Stemmed Words:", stemmed_words)


In [35]:
# import re
# import snowballstemmer

# # Initialize the NepaliStemmer instance
# nepali_stemmer = snowballstemmer.NepaliStemmer()

# def preprocess_text(text):
#     if isinstance(text, str):
#         # Remove punctuation and extra spaces
#         text = re.sub(r'[^\w\s]', '', text)
#         return text.strip()
#     return text

# def stem_text(text):
#     text = preprocess_text(text)
#     if isinstance(text, str):
#         return " ".join(nepali_stemmer.stemWords(text.split()))
#     return text

# # Apply the preprocessing and stemming to the 'title3' column
# data['data4'] = data['title3'].apply(stem_text)

# # Display results
# print(data[['title3', 'data4']].head(20))
# data.head(20)

In [36]:
data.columns

Index(['Title', 'class', 'Title2', 'tokens1', 'title3'], dtype='object')

In [37]:
x = data['title3']
y = data['class']

In [38]:
x

0        ['ओबामाले',,'अल्टोन',,'स्टर्लिङको',,'अन्त्येष्...
1        ['विश्वकप',,'फाइनल',,'फ्रान्सका',,'राष्ट्रपति'...
2                           ['भाषा',,'विस्तारमाथि',,'बहस']
3                 ['दैनिक',,'ज्याला',,'आठ',,'किलो',,'धान']
4        ['केबीसी',,'को',,'पहिलो',,'एपिसोडको',,'प्रोमो'...
                               ...                        
19509            ['द',,'बडी',,'सपको',,'उत्पादन',,'बजारमा']
19510    ['बाँकेमा',,'तीन',,'हजार',,'मेट्रिक',,'टन',,'म...
19511    ['यो',,'सबै',,'कहाँ',,'जित्यो',,'अर्को',,'न्या...
19512    ['‘',,'पीडितको',,'उद्धार',,'पहिलो',,'प्राथमिकत...
19513      ['बेलबटमको',,'पुरानो',,'फेसन',,'नयाँ',,'लुक्स']
Name: title3, Length: 19514, dtype: object

In [39]:
y

0        0
1        1
2        1
3        1
4        1
        ..
19509    1
19510    1
19511    0
19512    1
19513    1
Name: class, Length: 19514, dtype: int64

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [41]:
x_train.shape

(13659,)

In [42]:
x_test.shape

(5855,)

In [43]:
x_train.head()

2952     ['कंग्रेसमा',,'पल',,'र्यान',,'हुँदा',,'न्यान्स...
4898     ['‘',,'कार्बन',,'उत्सर्जन',,'शून्यमा',,'झार्ने...
12216    ['फर्ग्युसनमा',,'ठगको',,'वार्षिकोत्सव',,'हिंसा...
19164    ['कोरी',,'बुकरले',,'नयाँ',,'मुस्लिम',,'प्रतिबन...
10872    ['पीसी',,'पुलिस',,'को',,'मृत्यु',,'ट्रम्पको',,...
Name: title3, dtype: object

In [44]:
x_test.head()

2636     ['बर्नी',,'स्यान्डर्सले',,'भर्खरै',,'आफ्नो',,'...
7018     ['अमेरिकामा',,'मुस्लिम',,'प्रशिक्षण',,'शिविरको...
10853    ['साइक्लिङमा',,'आर्मी',,'र',,'एपीएफलाई',,'स्वर...
439      ['REDUX',,'डीप',,'स्टेट',,'बनाम',,'डोनाल्ड',,'...
19084    ['हेर्नुहोस्',,'इबोनी',,'म्यागजिन',,'सम्पादकले...
Name: title3, dtype: object

In [45]:
# here i built different file which contain only testing data
x_test.to_csv('testing_data.csv', index=False)

In [46]:
# similarly building file for training data
x_train.to_csv('training_data.csv', index=False)

# Vectorizing the text data into numerical


In [48]:
# #  here due to the nepali datasets the system doest tfidf the data.
# # so for that i have to convert the data into numerical featues using countvectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(ngram_range=(1,2)).fit(x_train)
# x_train_vectorized = vectorizer.transform(x_train)
# print(x_train_vectorized)
# x_train_vectorized.shape

In [49]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
# x_train_vectorized = vectorizer.fit_transform(x_train)

# Tfidf vectorization

In [51]:
vectorization = TfidfVectorizer() 

In [52]:
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic regression

In [54]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [55]:
pred_lr=LR.predict(xv_test)

In [56]:
LR.score(xv_test, y_test)

0.9251921434671221

In [57]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      2990
           1       0.92      0.93      0.92      2865

    accuracy                           0.93      5855
   macro avg       0.93      0.93      0.93      5855
weighted avg       0.93      0.93      0.93      5855



In [58]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    # Create a DataFrame from the input news
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)

    # Apply preprocessing using wordopt (assuming it's a defined function)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 

    # Transform the text using vectorization
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    # Perform prediction using Logistic Regression
    pred_LR = LR.predict(new_xv_test)

    # Output the prediction result
    return print("\n\nLogistic Regression Prediction: {}".format(output_label(pred_LR[0])))


In [113]:
news = "ओबामाले अल्टोन स्टर्लिङको अन्त्येष्टिमा प्रतिन...	"
manual_testing(news)




Logistic Regression Prediction: Fake News
